In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
import re
import unicodedata as ud

cid = # use your own if you have one
secret = # use your own if you have one

DaBeebs='spotify:artist:3WrFJ7ztbogyGnTHbHJFl2'
AbbyWoad='spotify:album:0ETFjACtuP2ADo6LFhL6HN'
Bowie='spotify:artist:0oSGxfWSnnOXhD2fKuz2Gy'
Stones='spotify:artist:22bE4uQ6baNwSHPVcDxLCe'
Dylan='spotify:artist:74ASZWbe4lXaubB36ztrGX'
Brown='spotify:artist:7GaxyUddsPok8BuhxN6OUW'
Who='spotify:artist:67ea9eGLXYMsO2eYQRui3w'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager, requests_timeout=60, retries=2)

In [2]:
def get_album_tracks(uri_info):
    uri = []
    track = []
    duration = []
    explicit = []
    album=[]
    df=pd.DataFrame()
    offset=0
    try:
        for i in range(5):
            chunk = sp.album_tracks(uri_info, limit=50, offset=offset, market='US')
            dt = pd.DataFrame(chunk)
            df=pd.concat([df,dt])
            offset+=50
    
        for i, x in df['items'].items():
            uri.append(x['uri'])
            track.append(x['name'])
            duration.append(x['duration_ms'])
            explicit.append(x['explicit'])
            album.append(sp.album(uri_info)['name'])

        df2 = pd.DataFrame({
        'uri':uri,
        'track':track,
        'duration_ms':duration,
        'explicit':explicit,
        'album':album
        })

        return df2
    except requests.exceptions.Timeout:
        print("Timeout occurred")
        return
    
def get_album_uris(uri_info):
    album_uri=[]
    album=[]
    date=[]
    offset=0
    df=pd.DataFrame()
    for i in range(5):
        chunk=sp.artist_albums(uri_info, limit=50, offset=offset)
        dt=pd.DataFrame(chunk)
        df=pd.concat([df,dt])
        offset+=50
    
    for i, x in df['items'].items():
            album_uri.append(x['uri'])
            album.append(x['name'])
            date.append(x['release_date'])
    df2=pd.DataFrame({
        'album_uri':album_uri,
        'album':album,
        'date':date
    })
    
    return df2

In [27]:
def _replace_all_brs(soup, attr='\n'):
    for tag in soup.find_all('br'):
        tag.replaceWith(attr)
    return soup

def geniusify(track_name):
    track_name=ud.normalize('NFKD', track_name.get_text()).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return re.sub(r'\s?-\s?|\s+','-',re.sub(r"[^\w\s\-]|\s[\-]\s.*$",'',re.sub(r"\:|\s*[\\\/]\s*"," ",track_name))).rstrip()

#function to scrape lyrics from genius
def scrape_lyrics(artistname, songname):
    songname2=geniusify(songname)
    artistname2 = re.sub(r'\s+','-',artistname)
    print(artistname2+'-'+songname2)
    page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics1= html.find_all("div", class_="Lyrics__Container-sc-1ynbvzw-10 cvsIWi")
    lyrics2=[_replace_all_brs(lyrics1[i]) for i in range(len(lyrics1))]
    if lyrics1:
        lyrics = ""
        for i in range(len(lyrics1)):
            lyrics+=lyrics2[i].get_text()
        lyrics=(re.sub(r"\n{2}",r"\n",lyrics))
    else:
        lyrics = None
        print('LYRICS NOT FOUND!')
    return lyrics

#function to attach lyrics onto data frame
#artist_name should be inserted as a string
def lyrics_onto_frame(df1, artist_name):
    for i,song_name in enumerate(df1['track']):
        text = scrape_lyrics(artist_name, song_name)
        df1.loc[i, 'lyrics'] = text
    return df1

In [34]:
import datetime
def genius_spotify_masher(artist_token, year_until=None):
    df=get_album_uris(artist_token)
    df['date']=pd.to_datetime(df['date'])
    # add a column for Year
    df['year'] = df['date'].dt.year
    if year_until:
        df=df[df['year']<=year_until]
    df=pd.concat([get_album_tracks(uri)for uri in df['album_uri']]).merge(df)
    df['genius_track']=df['track'].apply(lambda x: geniusify(x))
    df=df.sort_values(by='date').drop_duplicates(subset='genius_track', keep="first").drop('genius_track', axis=1).reset_index(drop=True)
    df=lyrics_onto_frame(df, sp.artist(artist_token)['name'])
    return df.sort_values(by='track').dropna().reset_index(drop=True)

In [36]:
genius_spotify_masher(Who).to_csv('TheWho.csv')

The-Who-A-Legal-Matter
The-Who-Its-Not-True
The-Who-The-Girls-I-Couldve-Had
The-Who-My-Generation
The-Who-Im-A-Man
The-Who-Out-In-The-Street
The-Who-Daddy-Rolling-Stone
The-Who-Circles
The-Who-Anyway-Anyhow-Anywhere
The-Who-Instant-Party-Mixture
The-Who-Anytime-You-Want-Me
The-Who-Motoring
The-Who-Love-Is-Like-A-Heat-Wave
The-Who-Shout-And-Shimmy
The-Who-As-Children-We-Grew
LYRICS NOT FOUND!
The-Who-Lubie-Come-Back-Home
The-Who-Bald-Headed-Woman
The-Who-I-Cant-Explain
The-Who-The-Kids-Are-Alright
The-Who-The-Goods-Gone
The-Who-I-Dont-Mind
The-Who-Leaving-Here
The-Who-Sunrise
The-Who-Much-Too-Much
The-Who-Please-Please-Please
The-Who-The-Ox
LYRICS NOT FOUND!
The-Who-My-Own-Love
LYRICS NOT FOUND!
The-Who-La-La-La-Lies
The-Who-Instant-Party
LYRICS NOT FOUND!
The-Who-Dont-Look-Away
The-Who-Disguises
The-Who-Doctor-Doctor
The-Who-In-The-City
The-Who-Ive-Been-Away
The-Who-Barbara-Ann
The-Who-Bucket-T
The-Who-Happy-Jack
The-Who-Cobwebs-And-Strange
LYRICS NOT FOUND!
The-Who-Batman
The-Who-A-Qu

LYRICS NOT FOUND!
The-Who-Miss-You
LYRICS NOT FOUND!
The-Who-What-You-Need-Dont-Change
LYRICS NOT FOUND!
The-Who-King-Of-Rock
LYRICS NOT FOUND!
The-Who-Why-I-Sing-The-Blues-Dont-Answer-The-Door-Rock-Me-Baby
LYRICS NOT FOUND!
The-Who-State-Of-Shock-Its-Only-Rock-N-Roll
LYRICS NOT FOUND!
The-Who-The-Reflex
LYRICS NOT FOUND!
The-Who-Forever-Young
LYRICS NOT FOUND!
The-Who-Hide-And-Seek
LYRICS NOT FOUND!
The-Who-Union-Of-The-Snake
LYRICS NOT FOUND!
The-Who-Every-Time-You-Go-Away
LYRICS NOT FOUND!
The-Who-Kids-Wanna-Rock
LYRICS NOT FOUND!
The-Who-Summer-Of-69
LYRICS NOT FOUND!
The-Who-Sunday-Bloody-Sunday
LYRICS NOT FOUND!
The-Who-Don-Quixote
LYRICS NOT FOUND!
The-Who-Bad
LYRICS NOT FOUND!
The-Who-Wouldnt-It-Be-Nice
LYRICS NOT FOUND!
The-Who-Good-Vibrations
LYRICS NOT FOUND!
The-Who-Surfin-USA
LYRICS NOT FOUND!
The-Who-Money-For-Nothing
LYRICS NOT FOUND!
The-Who-Sultans-Of-Swing
LYRICS NOT FOUND!
The-Who-Madison-Blues
LYRICS NOT FOUND!
The-Who-Bohemian-Rhapsody-Radio-Gaga
LYRICS NOT FOUND!


LYRICS NOT FOUND!
The-Who-We-Havent-Turned-Around
LYRICS NOT FOUND!
The-Who-Dont-Rain-On-My-Parade
LYRICS NOT FOUND!
The-Who-Because
LYRICS NOT FOUND!
The-Who-Dancing-In-The-Street
The-Who-Medley-Shakin-All-Over-Spoonful
LYRICS NOT FOUND!
The-Who-Relay
The-Who-Boris-The-Spider-Radio-1-Jingle
The-Who-My-Generation-Radio-1-Jingle
The-Who-Just-You-And-Me-Darling
The-Who-Good-Lovin
The-Who-One-Way-Out
LYRICS NOT FOUND!
The-Who-My-Cherie-Amour
LYRICS NOT FOUND!
The-Who-Ive-Seen-All-Good-People-a-Your-Move-b-All-Good-People
LYRICS NOT FOUND!
The-Who-It-Wouldnt-Have-Made-Any-Difference
LYRICS NOT FOUND!
The-Who-Roundabout
LYRICS NOT FOUND!
The-Who-America
LYRICS NOT FOUND!
The-Who-Looking-at-You
LYRICS NOT FOUND!
The-Who-River
LYRICS NOT FOUND!
The-Who-Slip-Away
LYRICS NOT FOUND!
The-Who-Waiting-For-The-Man
LYRICS NOT FOUND!
The-Who-Reelin-In-The-Years
LYRICS NOT FOUND!
The-Who-Dear-Jill
LYRICS NOT FOUND!
The-Who-Small-Time-Blues
LYRICS NOT FOUND!
The-Who-Go-All-The-Way
LYRICS NOT FOUND!
The-

LYRICS NOT FOUND!
The-Who-The-Happening
LYRICS NOT FOUND!
The-Who-Cleos-Mood
LYRICS NOT FOUND!
The-Who-These-Arms-of-Mine
LYRICS NOT FOUND!
The-Who-Sunny-Afternoon
LYRICS NOT FOUND!
The-Who-Judy-in-Disguise-with-Glasses
LYRICS NOT FOUND!
The-Who-Cant-Get-Enough-Of-Your-Love-Babe
LYRICS NOT FOUND!
The-Who-All-Day-and-All-of-the-Night
LYRICS NOT FOUND!
The-Who-Get-Up-Offa-That-Thing
LYRICS NOT FOUND!
The-Who-Signed-Sealed-Delivered-Im-Yours
LYRICS NOT FOUND!
The-Who-Stuck-In-The-Middle-With-You
LYRICS NOT FOUND!
The-Who-There-She-Goes
LYRICS NOT FOUND!
The-Who-The-Changingman
LYRICS NOT FOUND!
The-Who-Kung-Fu-Fighting
LYRICS NOT FOUND!
The-Who-Sit-Down
LYRICS NOT FOUND!
The-Who-Mistress-Mabel
LYRICS NOT FOUND!
The-Who-Cool-For-Cats
LYRICS NOT FOUND!
The-Who-Monster
LYRICS NOT FOUND!
The-Who-Jackie-Wilson-Said-Im-In-Heaven-When-You-Smile
LYRICS NOT FOUND!
The-Who-Mr-Big-Stuff
LYRICS NOT FOUND!
The-Who-I-Want-You-Back
LYRICS NOT FOUND!
The-Who-Reach-Out-Ill-Be-There
LYRICS NOT FOUND!
The-W

LYRICS NOT FOUND!
The-Who-Wanted-Dead-Or-Alive
LYRICS NOT FOUND!
The-Who-Rock-This-Town
LYRICS NOT FOUND!
The-Who-I-Drove-All-Night
LYRICS NOT FOUND!
The-Who-An-American-Dream
LYRICS NOT FOUND!
The-Who-Nineteen-Forever
LYRICS NOT FOUND!
The-Who-Drivers-Seat
LYRICS NOT FOUND!
The-Who-Name-and-Number
LYRICS NOT FOUND!
The-Who-Voyage-voyage
LYRICS NOT FOUND!
The-Who-The-Other-Side-of-the-Sun
LYRICS NOT FOUND!
The-Who-Satisfied
LYRICS NOT FOUND!
The-Who-Ode-To-Billie-Joe
LYRICS NOT FOUND!
The-Who-Overkill
LYRICS NOT FOUND!
The-Who-My-Generation-Remix
The-Who-Naked-Eye-Lets-See-Action-My-Generation-Medley
LYRICS NOT FOUND!
The-Who-Bad-to-Me
LYRICS NOT FOUND!
The-Who-When-a-Man-Loves-a-Woman
LYRICS NOT FOUND!
The-Who-The-Legend-Of-Xanadu
LYRICS NOT FOUND!
The-Who-A-World-Without-Love-Mono
LYRICS NOT FOUND!
The-Who-Foot-Tapper
LYRICS NOT FOUND!
The-Who-Sunshine-Superman
LYRICS NOT FOUND!
The-Who-123
LYRICS NOT FOUND!
The-Who-Im-Into-Something-Good
LYRICS NOT FOUND!
The-Who-Summer-in-the-City


...

LYRICS NOT FOUND!
The-Who-Spooky-Single-Version
LYRICS NOT FOUND!
The-Who-Somethings-Got-A-Hold-On-Me
LYRICS NOT FOUND!
The-Who-You-Never-Can-Tell
LYRICS NOT FOUND!
The-Who-Long-Tall-Sally-The-Thing
LYRICS NOT FOUND!
The-Who-Reminiscing
LYRICS NOT FOUND!
The-Who-Lets-Stick-Together
LYRICS NOT FOUND!
The-Who-Waterloo
LYRICS NOT FOUND!
The-Who-Dont-Stop-Me-Now
LYRICS NOT FOUND!
The-Who-What-A-Wonderful-World
LYRICS NOT FOUND!
The-Who-The-Power-Of-Love
LYRICS NOT FOUND!
The-Who-Dont-Be-Afraid-Of-The-Dark
LYRICS NOT FOUND!
The-Who-Head-Over-Heels
LYRICS NOT FOUND!
The-Who-Mony-Mony
LYRICS NOT FOUND!
The-Who-Shouldve-Known-Better
LYRICS NOT FOUND!
The-Who-Like-The-Way-I-Do
LYRICS NOT FOUND!
The-Who-Jailbreak
LYRICS NOT FOUND!
The-Who-Midnight-Rider
LYRICS NOT FOUND!
The-Who-You-Wear-It-Well
LYRICS NOT FOUND!
The-Who-Love-Is-A-Battlefield
LYRICS NOT FOUND!
The-Who-Fall-At-Your-Feet
LYRICS NOT FOUND!
The-Who-Tuesdays-Dead
LYRICS NOT FOUND!
The-Who-Freeze-Frame
LYRICS NOT FOUND!
The-Who-Mama-T

LYRICS NOT FOUND!
The-Who-Hard-To-Handle
LYRICS NOT FOUND!
The-Who-Pass-The-Dutchie
LYRICS NOT FOUND!
The-Who-Big-Yellow-Taxi
LYRICS NOT FOUND!
The-Who-The-First-Cut-Is-The-Deepest
LYRICS NOT FOUND!
The-Who-Little-Wing
LYRICS NOT FOUND!
The-Who-Dancing-In-The-Moonlight
LYRICS NOT FOUND!
The-Who-Billie-Jean
LYRICS NOT FOUND!
The-Who-I-Can-Hear-Music
LYRICS NOT FOUND!
The-Who-I-Shot-The-Sheriff
LYRICS NOT FOUND!
The-Who-We-Can-Work-It-Out
LYRICS NOT FOUND!
The-Who-Statesboro-Blues
LYRICS NOT FOUND!
The-Who-Crossroads
LYRICS NOT FOUND!
The-Who-Searching
LYRICS NOT FOUND!
The-Who-Tainted-Love
LYRICS NOT FOUND!
The-Who-Revolution-Rehearsal
LYRICS NOT FOUND!
The-Who-Mozart-Sonata-In-C-Major-1st-Movement
LYRICS NOT FOUND!
The-Who-de-Falla-Ritual-Fire-Dance
LYRICS NOT FOUND!
The-Who-Brian-Jones-Introduction-Of-Julius-Katchen
LYRICS NOT FOUND!
The-Who-Yer-Blues-Take-2
LYRICS NOT FOUND!
The-Who-Warmup-Jam
LYRICS NOT FOUND!
The-Who-Corinna
LYRICS NOT FOUND!
The-Who-Leaving-Trunk
LYRICS NOT FOUND!

LYRICS NOT FOUND!
The-Who-Over-The-Hill
LYRICS NOT FOUND!
The-Who-Pink-Moon
LYRICS NOT FOUND!
The-Who-Better-Together
LYRICS NOT FOUND!
The-Who-Here-Comes-The-Breeze
LYRICS NOT FOUND!
The-Who-Sometimes
LYRICS NOT FOUND!
The-Who-I-Feel-It-All
LYRICS NOT FOUND!
The-Who-Rinse-Me-Down
LYRICS NOT FOUND!
The-Who-Godless
LYRICS NOT FOUND!
The-Who-Butterfly
LYRICS NOT FOUND!
The-Who-A-Change-Would-Do-You-Good
LYRICS NOT FOUND!
The-Who-Walkaway
LYRICS NOT FOUND!
The-Who-Pale-Blue-Eyes
LYRICS NOT FOUND!
The-Who-Thirteen
LYRICS NOT FOUND!
The-Who-No-Rain
LYRICS NOT FOUND!
The-Who-Seaside
LYRICS NOT FOUND!
The-Who-Down-To-Zero
LYRICS NOT FOUND!
The-Who-Fade-Into-You
LYRICS NOT FOUND!
The-Who-What-I-Got
LYRICS NOT FOUND!
The-Who-Pink-Houses
LYRICS NOT FOUND!
The-Who-Burn-One-Down
LYRICS NOT FOUND!
The-Who-First-Day-Of-My-Life
LYRICS NOT FOUND!
The-Who-Diggin-A-Hole
LYRICS NOT FOUND!
The-Who-Radioactive
LYRICS NOT FOUND!
The-Who-All-The-Small-Things
LYRICS NOT FOUND!
The-Who-Buddy-Holly
LYRICS NOT F

LYRICS NOT FOUND!
The-Who-Heroin
LYRICS NOT FOUND!
The-Who-No-Woman-No-Cry
LYRICS NOT FOUND!
The-Who-In-Memory-Of-Elizabeth-Reed
LYRICS NOT FOUND!
The-Who-Louise
LYRICS NOT FOUND!
The-Who-Sneakin-Sally-Through-The-Alley
LYRICS NOT FOUND!
The-Who-Come-On-Eileen
LYRICS NOT FOUND!
The-Who-Mona
LYRICS NOT FOUND!
The-Who-Angie
LYRICS NOT FOUND!
The-Who-Amanda
LYRICS NOT FOUND!
The-Who-My-Girl-Josephine-AKA-Hello-Josephine
LYRICS NOT FOUND!
The-Who-Ruby-Tuesday
LYRICS NOT FOUND!
The-Who-Rosie
LYRICS NOT FOUND!
The-Who-Bess-You-Is-My-Woman-Now
LYRICS NOT FOUND!
The-Who-Angelica
LYRICS NOT FOUND!
The-Who-Peg
LYRICS NOT FOUND!
The-Who-You-Are-The-Sunshine-Of-My-Life
LYRICS NOT FOUND!
The-Who-Photograph
LYRICS NOT FOUND!
The-Who-Youre-The-One-That-I-Want
LYRICS NOT FOUND!
The-Who-Bobby-Brown-Goes-Down
LYRICS NOT FOUND!
The-Who-Hot-Stuff
LYRICS NOT FOUND!
The-Who-Is-This-Love
LYRICS NOT FOUND!
The-Who-Please-Mr-Postman
LYRICS NOT FOUND!
The-Who-Boys-Dont-Cry
LYRICS NOT FOUND!
The-Who-Out-Of-Time


LYRICS NOT FOUND!
The-Who-Cortez-The-Killer
LYRICS NOT FOUND!
The-Who-Colour-My-World
LYRICS NOT FOUND!
The-Who-Cover-Of-The-Rolling-Stone
LYRICS NOT FOUND!
The-Who-Misty-Mountain-Hop
LYRICS NOT FOUND!
The-Who-Voodoo-Child-Slight-Return
LYRICS NOT FOUND!
The-Who-Chance-Upon-You
LYRICS NOT FOUND!
The-Who-Please-Welcome-Stillwater
LYRICS NOT FOUND!
The-Who-Your-Mom-Kind-Of-Freaked-Me-Out
LYRICS NOT FOUND!
The-Who-Mona-Lisas-And-Mad-Hatters
LYRICS NOT FOUND!
The-Who-Piggyback-Ride-San-Diego
LYRICS NOT FOUND!
The-Who-This-Is-Penny-Lane
LYRICS NOT FOUND!
The-Who-Prefunction
LYRICS NOT FOUND!
The-Who-Cabin-By-The-River
LYRICS NOT FOUND!
The-Who-Dapple-Tree
LYRICS NOT FOUND!
The-Who-Cabin-In-The-Air
LYRICS NOT FOUND!
The-Who-Dear-Peggy
LYRICS NOT FOUND!
The-Who-Bye-Bye-Now
LYRICS NOT FOUND!
The-Who-Cabin
LYRICS NOT FOUND!
The-Who-Moody
LYRICS NOT FOUND!
The-Who-Morocco
LYRICS NOT FOUND!
The-Who-Aurora
LYRICS NOT FOUND!
The-Who-Mood-Swing
LYRICS NOT FOUND!
The-Who-The-Teenager
LYRICS NOT FOUND

In [37]:
whodat=pd.read_csv('TheWho.csv', index_col="Unnamed: 0")
whodat

,uri,track,duration_ms,explicit,album,album_uri,date,year,lyrics
0,spotify:track:5IO4w8NjPM5IHzAYtvX3T9,(I'm A) Road Runner - Live,194306,False,Who's Next,spotify:album:7otIoNh7kZgNoiNPauPaHU,1971-08-14,1971,"I'm a roadrunner, honey\nI'm a roadrunner, hon..."
1,spotify:track:088tGxSfC7CSLL2Uv2bUv6,(Love Is Like A) Heat Wave,159426,False,My Generation (50th Anniversary / Super Deluxe),spotify:album:0ooEHMMu3yiwnFwNYHcqwX,1965-12-03,1965,"Ooh ooh ooh ooh ooh, ooooh\nOoh ooh ooh ooh oo..."
2,spotify:track:4wLjoh8cf2dkHcG6I4aKiO,1921,150662,False,Woodstock Festival 1969,spotify:album:16g7cO2zMye8CiBx1SUL6t,1969-01-01,1969,Got a feeling '21\nIs going to be a good year\...
3,spotify:track:6eLREMcx73W3rm7ZDMJ6fM,1951 / What About The Boy?,169053,False,Tommy,spotify:album:5cT7ee1sy2oEbFalP4asS4,1969-05-23,1969,Nora:\nGot a feeling fifty one\nIs going to be...
4,spotify:track:1juPlxe83VSPEJQ7HyfbWK,5: 15,293009,False,The Who Hits 50,spotify:album:4MWzCje4KFt8oi2aHbhKUN,2014-10-27,2014,[Intro]\nWhy should I care?\nWhy should I care...
...,...,...,...,...,...,...,...,...,...
263,spotify:track:4hoyvWwGi3QXn7c47SLuqI,You,281600,False,Face Dances,spotify:album:2zLGQD3h1VG2tKzJCdAAyR,1981-03-16,1981,[Verse 1]\nYou – you are wasting my time\nWe a...
264,spotify:track:58RWYT5MGo8e4Y3WAZHh9g,You Better You Bet,337066,False,Face Dances,spotify:album:2zLGQD3h1VG2tKzJCdAAyR,1981-03-16,1981,"[Intro]\nYou better you better you bet, ooh\nY..."
265,spotify:track:2vh8pldwN9ntIEbgZ3Wgbc,You Stand By Me,96040,False,Endless Wire,spotify:album:2EQYiDShXSFZvryFM61Vv9,2006-01-01,2006,[Chorus]\nWhen I'm in trouble\nYou stand by me...
266,spotify:track:3xzFnSWKyzrmamAm41tpsR,Young Man Blues,380455,False,"Old England, New England",spotify:album:3477ClRrQJHN3OHM9G5CGk,1970-01-01,1970,Stage banter on Deluxe Edition only\n[Pete Tow...


In [38]:
who_lyrics="\n\n".join(whodat['lyrics'].astype(str).values)
print(who_lyrics)

I'm a roadrunner, honey
I'm a roadrunner, honey
And you can't keep up with me
Ooh, I'm a roadrunner, honey
And you can't keep up with me
I'm a, you and me
I'm a roadrunner, honey
And you can't keep up with me
Oh yeah, I'm a roadrunner, honey
And you can't keep up with me
I'm gonna show you
Baby, baby, you and me

Ooh ooh ooh ooh ooh, ooooh
Ooh ooh ooh ooh ooh, ooooh
Aah aah aah aah
Heat wave
Heat wave
Whenever I'm with you
Something inside
Starts to burn deep
And my heart's filled with fire
Could be that I'm very sentimental
Or is this just the way love's supposed to be?
I got a heat wave
Burning in my heart
I can't keep from crying
Tearing me apart
Sometimes she calls my name
Yeah, yeah, yeah, I can't explain
I feel, yeah I feel
I feel this burning flame
This high blood pressure's got a hold on me
'Cause it's just the way love's supposed to be
I got a heat wave
Burning in my heart
I can't keep from crying
Tearing me apart
Oh yeah
Oh yeah
Oh yeah
Oh yeah
Just give me another chance
Thi

In [40]:
with open('who_lyrics.txt','w', encoding ='utf-8') as f:
    f.write(who_lyrics)

In [42]:
genius_spotify_masher(DaBeebs).to_csv('DaBeebs.csv')

The-Beatles-Boys
The-Beatles-A-Taste-Of-Honey
The-Beatles-Do-You-Want-To-Know-A-Secret
The-Beatles-Baby-Its-You
The-Beatles-PS-I-Love-You
The-Beatles-Love-Me-Do
The-Beatles-Please-Please-Me
The-Beatles-Ask-Me-Why
The-Beatles-Chains
The-Beatles-Anna-Go-To-Him
The-Beatles-Misery
The-Beatles-I-Saw-Her-Standing-There
The-Beatles-Twist-And-Shout
The-Beatles-Theres-A-Place
The-Beatles-Devil-In-Her-Heart
The-Beatles-I-Wanna-Be-Your-Man
The-Beatles-You-Really-Got-A-Hold-On-Me
The-Beatles-Hold-Me-Tight
The-Beatles-Roll-Over-Beethoven
The-Beatles-Please-Mister-Postman
The-Beatles-Little-Child
The-Beatles-Dont-Bother-Me
The-Beatles-All-My-Loving
The-Beatles-All-Ive-Got-To-Do
The-Beatles-It-Wont-Be-Long
The-Beatles-Not-A-Second-Time
The-Beatles-Money-Thats-What-I-Want
The-Beatles-Till-There-Was-You
The-Beatles-Cant-Buy-Me-Love
The-Beatles-Ill-Be-Back
The-Beatles-A-Hard-Days-Night
The-Beatles-I-Should-Have-Known-Better
The-Beatles-If-I-Fell
The-Beatles-Im-Happy-Just-To-Dance-With-You
The-Beatles-An

LYRICS NOT FOUND!
The-Beatles-Like-Dreamers-Do
The-Beatles-The-Sheik-Of-Araby
The-Beatles-Three-Cool-Cats
The-Beatles-Searchin
The-Beatles-Free-As-A-Bird
The-Beatles-We-Were-Four-Guys-Thats-All
The-Beatles-Thatll-Be-The-Day
The-Beatles-In-Spite-Of-All-The-Danger
The-Beatles-Sometimes-Id-BorrowThose-Still-Exist
LYRICS NOT FOUND!
The-Beatles-Hallelujah-I-Love-Her-So
The-Beatles-Youll-Be-Mine
The-Beatles-First-Of-All-It-Didnt-Do-A-Thing-Here
LYRICS NOT FOUND!
The-Beatles-My-Bonnie
The-Beatles-Aint-She-Sweet
The-Beatles-Cry-For-A-Shadow
LYRICS NOT FOUND!
The-Beatles-Brian-Was-A-Beautiful-GuyHe-Presented-Us-Well
LYRICS NOT FOUND!
The-Beatles-I-Secured-Them-A-Beatle-Drink-Even-Then
LYRICS NOT FOUND!
The-Beatles-Cayenne
LYRICS NOT FOUND!
The-Beatles-Leave-My-Kitten-Alone
The-Beatles-You-Know-What-To-Do
The-Beatles-Shout
The-Beatles-Boys-What-Was-I-Thinking
LYRICS NOT FOUND!
The-Beatles-Moonlight-Bay
The-Beatles-12-Bar-Original
The-Beatles-If-Youve-Got-Trouble
The-Beatles-That-Means-A-Lot
The-

In [43]:
beebs=pd.read_csv('DaBeebs.csv', index_col="Unnamed: 0")
beebs

,uri,track,duration_ms,explicit,album,album_uri,date,year,lyrics
0,spotify:track:2HvTGx5fzFGpHSyRNvXd9T,12 Bar Original - Anthology 2 Version,175080,False,Anthology 2,spotify:album:3Lf8VA23cMAl5imbABTZoo,1996-03-18,1996,"[Paul McCartney]\nOne, two, three, four!\n[Ins..."
1,spotify:track:3MksxREMIslA65ZsIZHqzH,"1822! - Live At The BBC For ""Pop Go The Beatle...",10360,False,Live At The BBC (Remastered),spotify:album:2EowTulHWqSY6QZfTDf5vW,1994-11-30,1994,"[John] This is a Dorsey Burnette number, broth..."
2,spotify:track:4uKyLqJHDjT6mdA22ofxuM,A Beginning (Take 4) / Don’t Pass Me By (Take 7),305973,False,The Beatles,spotify:album:1WMVvswNzB9i2UMh9svso5,2018-11-09,2018,[Spoken]\nThis is the introduction to Ringo's ...
3,spotify:track:18qhv7DD1sEpgWffovGnhn,A Day In The Life - Remix,330906,False,Sgt. Pepper's Lonely Hearts Club Band (Super D...,spotify:album:1ntNLgaYCFCkeW4flGYlY2,1967-05-26,1967,[Verse 1: John Lennon]\nI read the news today—...
4,spotify:track:5J2CHimS7dWYMImCHkEFaJ,A Hard Day's Night - Remastered 2009,154200,False,A Hard Day's Night (Remastered),spotify:album:6wCttLq0ADzkPgtRnUihLV,1964-07-10,1964,[Verse 1: John Lennon]\nIt's been a hard day's...
...,...,...,...,...,...,...,...,...,...
298,spotify:track:175jiRVJ6N6ayJQ1FXim57,You'll Be Mine - Anthology 1 Version,98960,False,Anthology 1,spotify:album:1pBBIxK5yURfbv8Xd5lta1,1995-11-20,1995,"[Verse 1]\nWell the stars, always shine\nYou'l..."
299,spotify:track:70HNt0eoBVqr4ss68U8x3B,You're Going To Lose That Girl - Remastered 2009,138666,False,Help! (Remastered),spotify:album:0PT5m6hwPRrpBwIHVnvbFX,1965-08-06,1965,"[Chorus]\nYou're gonna lose that girl (Yes, ye..."
300,spotify:track:4F1AgKpuFRMLEgtPETVwZk,You've Got To Hide Your Love Away - Remastered...,129120,False,Help! (Remastered),spotify:album:0PT5m6hwPRrpBwIHVnvbFX,1965-08-06,1965,[Verse 1]\nHere I stand head in hand\nTurn my ...
301,spotify:track:3FuRBetiFz4ijeLdGCxQ5Y,"Young Blood - Live At The BBC For ""Pop Go The ...",115546,False,Live At The BBC (Remastered),spotify:album:2EowTulHWqSY6QZfTDf5vW,1994-11-30,1994,[Verse 1]\nI saw her standing on the corner (Y...


In [45]:
with open('beebs.txt','w', encoding ='utf-8') as f:
    f.write("\n\n".join(beebs['lyrics'].astype(str).values))